## MODEL LIKE

In [35]:
# Load libriaries and functions.
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import tensorflow_probability as tfp


In [36]:
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
tfd = tfp.distributions

In [37]:
# Define helper functions.
scaler = StandardScaler()
detector = IsolationForest(n_estimators=1000, random_state=42) # (of outliers)
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

In [38]:
# Load data and keep only first six months due to drift.
data = pd.read_excel("AirQualityUCI.xlsx")
data = data[data["Date"] <= "2004-09-10"]
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [39]:
# Select columns and remove rows with missing values.
columns = ["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH", "CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
data = data[columns].dropna(axis=0)
data.head()

,PT08.S1(CO),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,AH,CO(GT),C6H6(GT),NOx(GT),NO2(GT)
0,1360.00,1056.25,1692.00,1267.50,13.60,0.757754,2.6,11.881723,166.0,113.0
1,1292.25,1173.75,1558.75,972.25,13.30,0.725487,2.0,9.397165,103.0,92.0
2,1402.00,1140.00,1554.50,1074.00,11.90,0.750239,2.2,8.997817,131.0,114.0
3,1375.50,1092.00,1583.75,1203.25,11.00,0.786713,2.2,9.228796,172.0,122.0
4,1272.25,1205.00,1490.00,1110.00,11.15,0.788794,1.6,6.518224,131.0,116.0


In [40]:
# Scale data to zero mean and unit variance.
X_t = scaler.fit_transform(data)

In [41]:
# Remove outliers.
is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier > 0),:]

In [42]:
# Restore frame.
dataset = pd.DataFrame(X_t, columns=columns)
dataset.head()

,PT08.S1(CO),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,AH,CO(GT),C6H6(GT),NOx(GT),NO2(GT)
0,1.076431,0.654281,0.326732,0.914442,-0.059398,0.174465,0.539939,0.235974,0.702437,0.635984
1,0.852125,1.047609,0.003029,0.166726,-0.066752,0.173570,0.532772,0.170882,0.285297,0.465359
2,1.215484,0.934632,-0.007295,0.424406,-0.101071,0.174257,0.535161,0.160420,0.470693,0.644109
3,1.127748,0.773953,0.063762,0.751730,-0.123133,0.175268,0.535161,0.166471,0.742165,0.709109
4,0.785909,1.152218,-0.163984,0.515576,-0.119456,0.175326,0.527994,0.095458,0.470693,0.660359


In [43]:
# Select labels for inputs and outputs.
inputs = ["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH"]
outputs = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]

In [44]:
# Define some hyperparameters.
n_epochs = 50
n_samples = dataset.shape[0]
n_batches = 10
batch_size = np.floor(n_samples/n_batches)
buffer_size = n_samples

In [45]:
# Define training and test data sizes.
n_train = int(0.7*dataset.shape[0])
n_train

2646

In [46]:
# Define dataset instance.
data = tf.data.Dataset.from_tensor_slices((dataset[inputs].values, dataset[outputs].values))
data = data.shuffle(n_samples, reshuffle_each_iteration=True)

In [47]:
# Define train and test data instances.
data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1)

In [48]:
# Define prior for regularization.
prior = tfd.Independent(
    tfd.Normal(loc=tf.zeros(len(outputs), dtype=tf.float64), scale=1.0),
    reinterpreted_batch_ndims=1)

In [49]:
# Define model instance.
model = tfk.Sequential([

    # input
    tfk.layers.InputLayer(
        input_shape=(len(inputs),),
    name="input"),
    
    # dense for inputs
    tfk.layers.Dense(
        10, 
    activation="relu", name="dense_1"),
    
    # dense for weights
    tfk.layers.Dense(
        tfp.layers.MultivariateNormalTriL.params_size(len(outputs)), # uncertainty in the parameters weights
    activation=None, name="distribution_weights"),
    
    # (declaration of the) posterior probability distribution structure
    tfp.layers.MultivariateNormalTriL(
        len(outputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_batches), # activity_regularizer acts as prior for the output layer
    name="output")

], name="model")


In [50]:
# Compile model.
model.compile(optimizer="adam", loss=neg_log_likelihood)

In [51]:
# Run training session.
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)

In [52]:
# Describe model.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 10)                70        
                                                                 
 distribution_weights (Dense  (None, 14)               154       
 )                                                               
                                                                 
 output (MultivariateNormalT  ((None, 4),              0         
 riL)                         (None, 4))                         
                                                                 
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


To account for uncertainty in parameter weights, the dense layers have to be exchanged with

- Flipout layers (``DenseFlipout``)
- Variational layers (``DenseVariational``)

Such a model has more parameters, since every weight is parametrized by normal distribution with non-shared mean and standard deviation. \
Weights will be resampled for different predictions.

In [53]:
tfp.layers.DenseFlipout(10, activation="relu", name="dense_1")

The default prior distribution over weights is `tfd.Normal(loc=0., scale=1.)` and can be adjusted using the ``kernel_prior_fn``

In [54]:
# Predict.
samples = 500
iterations = 10
test_iterator = tf.compat.v1.data.make_one_shot_iterator(data_test)
X_true, Y_true, Y_pred = np.empty(shape=(samples, len(inputs))), np.empty(shape=(samples, len(outputs))), np.empty(shape=(samples, len(outputs), iterations))

In [55]:
for i in range(samples):
    features, labels = test_iterator.get_next()
    X_true[i,:] = features
    Y_true[i,:] = labels.numpy()
    for k in range(iterations):
        Y_pred[i,:,k] = model.predict(features)

In [56]:
# Calculate mean and standard deviation.
Y_pred_m = np.mean(Y_pred, axis=-1)
Y_pred_s = np.std(Y_pred, axis=-1)
Y_pred_m, Y_pred_s

(array([[ 0.58517306,  0.25242546,  1.00271954,  0.90930641],
        [-0.44923552,  0.03306342, -0.80769113, -0.73458287],
        [ 0.52697573,  0.16782712,  0.6568934 ,  0.55450277],
        ...,
        [ 0.40314043,  0.27925446,  0.54816173,  0.57473505],
        [ 0.42385402,  0.05678811,  0.29256234,  0.45154649],
        [ 0.34659357,  0.26924554,  0.35932272,  0.31697732]]),
 array([[0.0610732 , 0.03898739, 0.38077967, 0.47161293],
        [0.95617658, 0.01516663, 0.96943129, 1.19629141],
        [0.21000538, 0.04471322, 0.54004473, 0.47347334],
        ...,
        [0.49005494, 0.03783741, 0.39207623, 0.44975531],
        [0.52693271, 0.02029337, 0.45461153, 0.47234292],
        [0.71652314, 0.02481016, 0.59870454, 0.63849257]]))

## DATA

In [2]:
import pandas as pd
df_en = pd.read_csv('energy_dataset.csv')
df_we = pd.read_csv('weather_features.csv')

In [9]:
[c for c in df_en.columns if not c.startswith('generation')]

['time',
 'forecast solar day ahead',
 'forecast wind offshore eday ahead',
 'forecast wind onshore day ahead',
 'total load forecast',
 'total load actual',
 'price day ahead',
 'price actual']

In [8]:
[c for c in df_en.columns if c.startswith('generation')]

['generation biomass',
 'generation fossil brown coal/lignite',
 'generation fossil coal-derived gas',
 'generation fossil gas',
 'generation fossil hard coal',
 'generation fossil oil',
 'generation fossil oil shale',
 'generation fossil peat',
 'generation geothermal',
 'generation hydro pumped storage aggregated',
 'generation hydro pumped storage consumption',
 'generation hydro run-of-river and poundage',
 'generation hydro water reservoir',
 'generation marine',
 'generation nuclear',
 'generation other',
 'generation other renewable',
 'generation solar',
 'generation waste',
 'generation wind offshore',
 'generation wind onshore']

In [14]:
df_we.describe()

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id
count,178396.000000,178396.000000,178396.000000,1.783960e+05,178396.000000,178396.00000,178396.000000,178396.000000,178396.000000,178396.000000,178396.000000,178396.000000
mean,289.618605,288.330442,291.091267,1.069261e+03,68.423457,2.47056,166.591190,0.075492,0.000380,0.004763,25.073292,759.831902
std,8.026199,7.955491,8.612454,5.969632e+03,21.902888,2.09591,116.611927,0.398847,0.007288,0.222604,30.774129,108.733223
min,262.240000,262.240000,262.240000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,200.000000
25%,283.670000,282.483602,284.650000,1.013000e+03,53.000000,1.00000,55.000000,0.000000,0.000000,0.000000,0.000000,800.000000
50%,289.150000,288.150000,290.150000,1.018000e+03,72.000000,2.00000,177.000000,0.000000,0.000000,0.000000,20.000000,800.000000
75%,295.150000,293.730125,297.150000,1.022000e+03,87.000000,4.00000,270.000000,0.000000,0.000000,0.000000,40.000000,801.000000
max,315.600000,315.150000,321.150000,1.008371e+06,100.000000,133.00000,360.000000,12.000000,2.315000,21.500000,100.000000,804.000000


In [13]:
df_en.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000
